In [ ]:
### Running questions
### Note: I will clean up stuff for sure (moving lines to functions, etc), this is more about logic 

In [ ]:
import cyvcf2
from pyfaidx import Fasta
import os
import argparse

In [ ]:
os.chdir('/Users/quinlan/u1264408/Git/spermseq')
os.listdir()

In [ ]:
A = '1SCS014.indels.vcf'
Atrust = '1SCS014.trustworthyindels.vcf'
B = 'K8J7003.indels.vcf'
C = 'UA6U012.indels.vcf'
D = 'YL99029.indels.vcf'
E = 'XH9R013.indels.vcf'

In [ ]:
genome = Fasta('/Users/quinlan/u1264408/human_g1k_v38_decoy_phix.fasta', as_raw=True) # look into as_RAW
# as_raw: optional parameter to specify whether to return sequences as a Sequence() object or as a raw string.
from cyvcf2 import VCF

In [ ]:
### Touching 5

vcf = VCF(Atrust)

failcount = 0
passcount = 0 

for v in vcf():
    valt = v.ALT[0]
    vref = v.REF
    if len(vref) > len(valt):
        vdiff = vref.replace(valt, '', 1)
    else: 
        vdiff = valt.replace(vref, '', 1)
    if vdiff == len(vdiff) * vdiff[0]:
        vdiff = vdiff[0]
    else:
        pass
    ### find the ins/del string difference between the ref and alt alleles
    
    genome_seq_left = genome[v.CHROM][v.POS-4:v.POS]
    genome_seq_right = genome[v.CHROM][v.POS:v.POS+4]

    if (genome_seq_left == len(genome_seq_left) * genome_seq_left[0]):
        genome_seq_left_use = genome_seq_left
    else:
        genome_seq_left_use = 'Left'
   
    if (genome_seq_right == len(genome_seq_right) * genome_seq_right[0]):
        genome_seq_right_use = genome_seq_right
    else:
        genome_seq_right_use = 'Right'

    ### ensure left and right flanks are homopolymers for failing 
    
    #print(genome_seq_left, genome_seq_right) 
    if 'N' in v.ALT[0]:
        #print('FAIL ', v.ALT[0])
        failcount += 1 
    elif vdiff  in genome_seq_left_use:
        #print( v.CHROM, v.POS, v.REF, v.ALT[0])            
        failcount += 1
        #print('vdiff in left')
    elif vdiff in genome_seq_right_use: 
        #print( v.CHROM, v.POS, v.REF, v.ALT[0])
        failcount += 1
        #print('vdiff in right')
    else:
        #print( v.CHROM, v.POS, v.REF, v.ALT[0])
        passcount += 1
        #print('PASS')
print(v.INFO.get('donor') + ":", 'Passcount is', passcount, 'and failcount is', failcount) 

 